In [27]:
def getLetter():
    input_letter = input()
    if input_letter.isalpha() and len(input_letter) == 1:
        return input_letter.lower()
    else:
        print('Error, try repeat')
        return getLetter()

In [7]:
class Letter():
    def __init__(self, letter, open_flag):
        self.letter = letter
        self.open_flag = open_flag
    
    def show(self):
        if self.open_flag:
            return self.letter
        else:
            return '*'

In [24]:
class Slovo():
    def __init__(self, word, num_attemts):
        self.word = word
        self.num_attemts = num_attemts

        letter_list = []
        for i in word:
            letter_list.append(Letter(i, False))
        self.letter_list = letter_list

    def show(self, show_word_flag, input_letter=None):
        new_word = ''
        if show_word_flag:
            return self.word
        else:
            num_attemts_flag = False
            for i in self.letter_list:
                if i.letter == input_letter:
                 num_attemts_flag = True
                 i.open_flag = True
                new_word += i.show()
            if not num_attemts_flag:
                 print('letter', input_letter, 'not in word')
                 self.num_attemts -=1
            return (new_word, self.num_attemts)
            


In [28]:
def game(word):
    num_attemts = 7
    slovo = Slovo(word, num_attemts)
    while num_attemts != 0:
        input_letter = getLetter()
        new_word, num_attemts = slovo.show(False, input_letter)
        print(new_word, 'attempts left:', num_attemts)

        if new_word == slovo.show(True):
            print('You win')
            break
    if num_attemts == 0:  
        print('Game over')

In [30]:
game('language')

l******* attempts left: 7
la***a** attempts left: 7
la***a** attempts left: 7
letter b not in word
la***a** attempts left: 6
lan**a** attempts left: 6
lang*ag* attempts left: 6
languag* attempts left: 6
languag* attempts left: 6
language attempts left: 6
You win
